In [1]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
from preprocessing import processing
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 
import statistics
import datetime
from dateutil.relativedelta import relativedelta
import random
random.seed(10)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Cell to change
sp = 'sp600'

In [3]:
# List of dowloaded articles
dir_path = "data/" + sp + "/text/"
tickers = os.listdir(dir_path)
#tickers.remove('.DS_Store') # for mac
print(len(tickers))

522


In [4]:
# Import earning calls

texts = []
list_ids = []                # list of the call ids in the same order as the list "texts"
list_tickers = []            # list of the tickers in the same order as the list "texts"
number_texts_per_stock = []

for ticker in tickers:
    dir_path = "data/"+sp+"/text/"+ticker+"/" 
    list_ids1 = os.listdir(dir_path)
    ticker1 = os.path.basename(os.path.dirname(dir_path))
    ticker1 = ticker1.split()
    texts1 = []
    for s in list_ids1:
        with open(dir_path + s, encoding = 'ISO-8859-1') as f:
            t = f.read()
            texts1.append(t)
    texts = texts + texts1
    list_ids = list_ids + list_ids1 
    list_tickers = list_tickers + ticker1*len(list_ids1)
    number_texts_per_stock.append(len(list_ids1))

In [5]:
# Get dates
data_path = "data/"+sp+"/ticker/id_to_date" 
with open(data_path,"rb") as file :
    id_to_date = pickle.load(file)

In [6]:
# Create dataframe with earning calls informations

# Earning call id
df1 = pd.DataFrame(data={'id': list_ids})

# Date
df1['date']=list_ids
df1=df1.replace({'date': id_to_date})
display(df1) #len : 21385 for sp500 & 19341 for SP600

# Ticker
df1['ticker']= list_tickers

# Text
df1['text']= texts

# Remove the non-identified dates
df1=df1[df1['date']!=df1['id']]

# Quarter earning call is reporting (1 quarter before the publication date)
df1['date'] = pd.to_datetime(df1['date'])
df1['lagged_date']=df1.date - pd.offsets.DateOffset(months=3)
df1['earning_call_quarter'] = df1['lagged_date'].dt.to_period('Q')
df1 = df1.drop(columns=['lagged_date'])

# Sort by ticker and date
df1= df1.sort_values(['ticker','date'], ascending=True)
display(df1)

,id,date
0,4070677,4070677
1,4446584,4446584
2,2187623,2187623
3,3970723,3970723
4,4507092,4507092
...,...,...
19336,3957575,3957575
19337,4505989,4505989
19338,4281462,4281462
19339,4304138,4304138


,id,date,ticker,text,earning_call_quarter
10514,72698,2008-04-17,AAN,Question-And-Answer Session\n \n\n \nOperator\...,2008Q1
10539,73635,2008-04-23,AAN,Question-and-Answer Session\n \n \n \nOperator...,2008Q1
10515,91435,2008-08-18,AAN,Question-and-Answer Session\n \nOperator\n \nT...,2008Q2
10537,121064,2009-02-17,AAN,Question-and-Answer Session\n \nOperator\n \n(...,2008Q4
10524,281669,2011-07-25,AAN,Question-and-Answer Session\n \nOperator\n \nL...,2011Q2
...,...,...,...,...,...
13391,4422733,2021-04-29,ZYXI,Question-and-Answer Session\n \nOperator\n \nW...,2021Q1
13395,4442847,2021-07-29,ZYXI,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
13401,4464718,2021-11-02,ZYXI,Question-and-Answer Session\n \nOperator\n \n[...,2021Q3
13394,4490333,2022-02-24,ZYXI,Question-and-Answer Session\n \nOperator\n \nT...,2021Q4


In [7]:
# Keep datas from 2019Q1 to last earning call (2022Q1)
start_date = '2019Q1'
end_date = '2022Q1'
mask = (df1['earning_call_quarter'] >= start_date)
df2 = df1.loc[mask]
display(df2)

# Compute n_quarters : Number of quarters between start_date and end_date
t = pd.to_datetime(end_date).to_period(freq='Q')-pd.to_datetime(start_date).to_period(freq='Q')
n_quarters = n_quarters = t.n+1 # n_quarters = 13 in our case

,id,date,ticker,text,earning_call_quarter
10504,4256794,2019-04-25,AAN,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
10533,4277628,2019-07-25,AAN,Question-and-Answer Session\n \nOperator\n \n[...,2019Q2
10525,4302423,2019-11-04,AAN,Question-and-Answer Session\n \nOperator\n \n[...,2019Q3
10528,4325838,2020-02-20,AAN,Question-and-Answer Session\n \nOperator\n \n[...,2019Q4
10510,4345482,2020-05-09,AAN,Question-and-Answer Session\n \nOperator\n \nW...,2020Q1
...,...,...,...,...,...
13391,4422733,2021-04-29,ZYXI,Question-and-Answer Session\n \nOperator\n \nW...,2021Q1
13395,4442847,2021-07-29,ZYXI,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
13401,4464718,2021-11-02,ZYXI,Question-and-Answer Session\n \nOperator\n \n[...,2021Q3
13394,4490333,2022-02-24,ZYXI,Question-and-Answer Session\n \nOperator\n \nT...,2021Q4


In [8]:
# Count number of texts per stock
texts_per_stock = df2[['id','ticker']].groupby(['ticker']).count().reset_index().rename(columns={'id': 'count'})
df_count = pd.DataFrame(texts_per_stock)

# Merge the initial DataFrame with the one containing the number of observation for each stock
df_merged = pd.merge(df2, df_count, on='ticker')
df_merged.reindex(columns=["id","date","earning_call_quarter","ticker","text","count"])

# Keep only the stocks that have n_quarters observations into the final DataFrame
df_final = df_merged[df_merged['count'] == n_quarters]
df_final = df_final.drop(columns = "count")

# Keep only the 100 first stocks
df_final = df_final[:1300]
display(df_final)

,id,date,ticker,text,earning_call_quarter
26,4260232,2019-05-05,AAT,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
27,4281104,2019-08-02,AAT,Question-and-Answer Session\n \nOperator\n \nT...,2019Q2
28,4301865,2019-11-03,AAT,Question-and-Answer Session\n \nOperator\n \nT...,2019Q3
29,4323613,2020-02-12,AAT,Question-and-Answer Session\n \nOperator\n \nT...,2019Q4
30,4341190,2020-04-29,AAT,Question-and-Answer Session\n \nOperator\n \nT...,2020Q1
...,...,...,...,...,...
1892,4420401,2021-04-21,PFBC,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
1893,4440488,2021-07-21,PFBC,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
1894,4461252,2021-10-21,PFBC,Question-and-Answer Session\n \nOperator\n \n[...,2021Q3
1895,4480666,2022-01-20,PFBC,Question-and-Answer Session\n \nOperator\n \nT...,2021Q4


In [9]:
# Save final dataframe
df_path = "data/df_final_" + sp
with open(df_path,"wb") as file : 
    pickle.dump(df_final, file, protocol=pickle.HIGHEST_PROTOCOL)

# Save ids in a csv file
df_final_ids = df_final['id']
df_final_ids.to_csv('df_100stocks_'+sp+'.csv')